# Data preparation

In [38]:
import numpy as np
import pandas as pd
import plotly.express as px

# Athletes

In [39]:
dataset_athlete = pd.read_json('../olympic_athletes.json')
dataset_athlete.head()
dataset_athlete.shape
dataset_athlete.isnull().sum()

athlete_url                 0
athlete_full_name           0
games_participations        0
first_game                 22
athlete_year_birth       2456
athlete_medals          60552
bio                     53062
dtype: int64

# Hotes

In [40]:
dataset_hosts = pd.read_xml('../olympic_hosts.xml')
dataset_hosts.head()
dataset_hosts.shape
dataset_hosts.columns
dataset_hosts.drop(['index'], axis=1, inplace=True)
dataset_hosts.columns
dataset_hosts.isnull().sum()

game_slug          0
game_end_date      0
game_start_date    0
game_location      0
game_name          0
game_season        0
game_year          0
dtype: int64

# Medailles

In [41]:
dataset_medals = pd.read_excel('../olympic_medals.xlsx')
dataset_medals.shape
dataset_medals.isnull().sum()
dataset_medals.duplicated().sum()
dataset_medals.columns
dataset_medals = dataset_medals.rename({'slug_game': 'game_slug'}, axis=1)
dataset_medals.columns
dataset_medals.loc[(dataset_medals.duplicated())]
dataset_medals.loc[(dataset_medals.game_slug == 'paris-1900') & (dataset_medals.discipline_title == 'Polo')]
dataset_medals.loc[(dataset_medals.game_slug == 'london-1908') & (dataset_medals.discipline_title == 'Polo')]
dataset_medals.loc[(dataset_medals.game_slug == 'london-1908') & (dataset_medals.discipline_title == 'Hockey') & (
        dataset_medals.event_title == 'hockey men')]
dataset_medals.participant_title.unique()
dataset_medals.drop(['Unnamed: 0', 'participant_title', 'athlete_url'], axis=1, inplace=True)
dataset_medals.tail()

,discipline_title,game_slug,event_title,event_gender,medal_type,participant_type,athlete_full_name,country_name,country_code,country_3_letter_code
21692,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,SILVER,Athlete,Viggo JENSEN,Denmark,DK,DEN
21693,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,BRONZE,Athlete,Alexandros Nikolopoulos,Greece,GR,GRE
21694,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,GOLD,Athlete,Viggo JENSEN,Denmark,DK,DEN
21695,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,SILVER,Athlete,Launceston ELLIOT,Great Britain,GB,GBR
21696,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,BRONZE,Athlete,Sotirios VERSIS,Greece,GR,GRE


# Results

In [42]:
dataset_results = pd.read_html('../olympic_results.html')[0]
dataset_results.head()
dataset_results.shape
dataset_results.isnull().sum()
dataset_results.duplicated().sum()
dataset_results.loc[(dataset_results.duplicated())]
dataset_results.value_unit.unique()
dataset_results.value_type.unique()
dataset_results.drop(['Unnamed: 0', 'athlete_url'], axis=1, inplace=True)
dataset_results = dataset_results.rename({'slug_game': 'game_slug'}, axis=1)
dataset_results.columns

Index(['discipline_title', 'event_title', 'game_slug', 'participant_type',
       'medal_type', 'athletes', 'rank_equal', 'rank_position', 'country_name',
       'country_code', 'country_3_letter_code', 'athlete_full_name',
       'value_unit', 'value_type'],
      dtype='object')

# Separate winter and summer games

In [43]:
jo_types = dataset_hosts[['game_slug', 'game_season', 'game_year']]
merged_hosts_results = dataset_results.merge(jo_types, on='game_slug')
merged_hosts_results.head()
merged_hosts_results.shape
merged_hosts_results.game_season.unique()

array(['Winter', 'Summer'], dtype=object)

In [44]:
summer_games_results = merged_hosts_results.loc[(merged_hosts_results.game_season == 'Summer')].copy()
winter_games_results = merged_hosts_results.loc[(merged_hosts_results.game_season == 'Winter')].copy()

In [45]:
summer_games_results.drop(['game_season'], axis=1, inplace=True)
summer_games_results.head()
summer_games_results.shape
summer_games_results.isnull().sum()
summer_games_results.duplicated().sum()
dopplers = summer_games_results.loc[(summer_games_results.duplicated())]
dopplers

,discipline_title,event_title,game_slug,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_full_name,value_unit,value_type,game_year
154361,Water Polo,Water Polo Women,antwerp-1920,GameTeam,NaN,NaN,NaN,1,Netherlands,NL,NED,NaN,2,SCORE,1920
155336,Sailing,8m mixed,stockholm-1912,GameTeam,NaN,NaN,NaN,5,Russian Federation,RU,RUS,NaN,0,POINTS,1912
157014,Shooting,trap 125 targets men,stockholm-1912,Athlete,GOLD,NaN,NaN,1,United States of America,US,USA,Jay Graham,96,POINTS,1912
157015,Shooting,trap 125 targets men,stockholm-1912,Athlete,SILVER,NaN,NaN,2,Germany,DE,GER,Alfred GOELDEL,94,POINTS,1912
157016,Shooting,trap 125 targets men,stockholm-1912,Athlete,BRONZE,NaN,NaN,3,Russian Federation,RU,RUS,Harry Blaus,91,POINTS,1912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161736,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161737,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161738,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161739,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900


# Calculate the total number of each type of medals for each games by country

In [46]:
summer_games_results.medal_type.unique()
summer_games_results['medal_type'] = summer_games_results['medal_type'].fillna('None')
summer_games_results['total_medals'] = summer_games_results['medal_type'].apply(lambda x: 0 if x == 'None' else 1)
summer_games_results['gold_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'GOLD' else 0)
summer_games_results['silver_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'SILVER' else 0)
summer_games_results['bronze_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'BRONZE' else 0)
summer_games_results
summer_games_results.country_name.unique()
historic_medalsbycountry = summer_games_results.groupby(['game_year', 'country_name']).agg(
    {'total_medals': 'sum', 'gold_medals': 'sum', 'silver_medals': 'sum', 'bronze_medals': 'sum'})
historic_medalsbycountry = historic_medalsbycountry.sort_values(by=['game_year', 'total_medals'],
                                                                ascending=[True, False]).reset_index()
historic_medalsbycountry

,game_year,country_name,total_medals,gold_medals,silver_medals,bronze_medals
0,1896,Greece,47,10,18,19
1,1896,United States of America,20,11,7,2
2,1896,Germany,13,6,5,2
3,1896,France,11,5,4,2
4,1896,Great Britain,7,2,3,2
...,...,...,...,...,...,...
2884,2020,"Virgin Islands, British",0,0,0,0
2885,2020,"Virgin Islands, US",0,0,0,0
2886,2020,Yemen,0,0,0,0
2887,2020,Zambia,0,0,0,0


# Calculate the total number of disciplines for each games by country

In [47]:
historic_sportsbycountry = summer_games_results.groupby(['game_year', 'country_name', 'discipline_title']).count()
historic_sportsbycountry
historic_sportsbycountry = historic_sportsbycountry[['event_title']].reset_index()
historic_sportsbycountry = historic_sportsbycountry.rename({'discipline_title': 'sports', 'event_title': 'epreuves'},
                                                           axis=1)
historic_sportsbycountry
historic_sportsbycountry = historic_sportsbycountry.groupby(['game_year', 'country_name']).agg({'sports': 'count'})
historic_sportsbycountry = historic_sportsbycountry.reset_index()
historic_sportsbycountry
historic_epreuvesbycountry = summer_games_results.groupby(['game_year', 'country_name', 'event_title']).count()
historic_epreuvesbycountry

historic_epreuvesbycountry = historic_epreuvesbycountry[['discipline_title']].reset_index()
historic_epreuvesbycountry = historic_epreuvesbycountry.rename(
    {'event_title': 'epreuves', 'discipline_title': 'participation'}, axis=1)
historic_epreuvesbycountry = historic_epreuvesbycountry.groupby(['game_year', 'country_name']).agg(
    {'epreuves': 'count'})
historic_epreuvesbycountry = historic_epreuvesbycountry.reset_index()
historic_epreuvesbycountry
historic_olympic_data = historic_medalsbycountry.merge(historic_sportsbycountry, on=['game_year', 'country_name'])
historic_olympic_data = historic_olympic_data.merge(historic_epreuvesbycountry, on=['game_year', 'country_name'])
historic_olympic_data = historic_olympic_data.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])
historic_olympic_data

,game_year,country_name,total_medals,gold_medals,silver_medals,bronze_medals,sports,epreuves
0,1896,Greece,47,10,18,19,10,39
1,1896,United States of America,20,11,7,2,3,17
2,1896,Germany,13,6,5,2,7,27
3,1896,France,11,5,4,2,7,20
4,1896,Great Britain,7,2,3,2,9,20
...,...,...,...,...,...,...,...,...
2884,2020,"Virgin Islands, British",0,0,0,0,1,2
2885,2020,"Virgin Islands, US",0,0,0,0,1,1
2886,2020,Yemen,0,0,0,0,2,2
2887,2020,Zambia,0,0,0,0,3,5


In [48]:
# Use summer_games_results to create columns that represent the number of participation at time t, and the number of medals won at time t-1 for each country and each game, merge the results with historic_olympic_data

game_part = summer_games_results.groupby(['country_name', 'game_year']).agg(
    {'total_medals': 'sum', 'gold_medals': 'sum', 'silver_medals': 'sum', 'bronze_medals': 'sum'})
game_part
game_p = game_part.reset_index()
game_p_france = game_p.loc[(game_p.country_name == 'France')]
game_p_france
import warnings

warnings.filterwarnings('ignore')
game_p_france = game_p.loc[(game_p.country_name == 'France')]
game_p_france['game_part'] = range(0, game_p_france.shape[0])
game_p_france['prec_game_medal'] = game_p_france['total_medals'].shift(1, fill_value=0)
game_p_france['prec_game_gold'] = game_p_france['gold_medals'].shift(1, fill_value=0)
game_p_france['prec_game_silver'] = game_p_france['silver_medals'].shift(1, fill_value=0)
game_p_france['prec_game_bronze'] = game_p_france['bronze_medals'].shift(1, fill_value=0)
game_p_france
game_p
liste_country = list(game_p.country_name.unique())
liste_country.remove('France')
liste_country
for country in liste_country:
    game_p_temp = game_p.loc[(game_p.country_name == country)]
    game_p_temp['game_part'] = range(0, game_p_temp.shape[0])
    game_p_temp['prec_game_medal'] = game_p_temp['total_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_gold'] = game_p_temp['gold_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_silver'] = game_p_temp['silver_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_bronze'] = game_p_temp['bronze_medals'].shift(1, fill_value=0)
    game_p_france = pd.concat([game_p_france, game_p_temp])
game_p_france.columns
game_p_france = game_p_france[
    ['game_year', 'country_name', 'total_medals', 'gold_medals', 'silver_medals', 'bronze_medals', 'game_part',
     'prec_game_medal', 'prec_game_gold', 'prec_game_silver', 'prec_game_bronze']]
game_p_france = game_p_france.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])
game_p_france
game_p_france = game_p_france[
    ['game_year', 'country_name', 'game_part', 'prec_game_medal', 'prec_game_gold', 'prec_game_silver',
     'prec_game_bronze']]
game_p_france
historic_olympic_data = pd.merge(historic_olympic_data, game_p_france, on=['game_year', 'country_name'])

historic_olympic_data = historic_olympic_data.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])

# Save the data to ../csv/olympic_data_cleaned.csv

In [49]:
historic_olympic_data.to_csv('../csv/olympic_data_cleaned.csv', index=False)

## Train Dense Neural Network

In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from keras import models, layers

In [51]:
# Load data
historic_olympic_data = pd.read_csv('../csv/olympic_data_cleaned.csv')

# Split data before 2020
data_before_2020 = historic_olympic_data[historic_olympic_data['game_year'] < 2020]
data_2020 = historic_olympic_data[historic_olympic_data['game_year'] == 2020]

# Features and target variables
features = ['sports', 'epreuves', 'game_part', 'prec_game_medal', 'prec_game_gold', 'prec_game_silver', 'prec_game_bronze']
target = ['gold_medals', 'silver_medals', 'bronze_medals']

# Define the number of epochs and batch size
epochs = 130
batch_size = 32

# Split features and targets
X_train = data_before_2020[features]
y_train = data_before_2020[target]
X_test = data_2020[features]
y_test = data_2020[target]
countries_2020 = data_2020['country_name']

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the dense neural network model
model_dense = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(3) # Output layer for gold, silver, and bronze medals
])

# Compile the model
model_dense.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history_dense = model_dense.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)

# Evaluate the model
loss_dense, mae_dense = model_dense.evaluate(X_test_scaled, y_test, verbose=0)

# Make predictions
y_pred_dense = model_dense.predict(X_test_scaled)

# Calculate additional metrics
mse_dense = mean_squared_error(y_test, y_pred_dense)
mae_dense = mean_absolute_error(y_test, y_pred_dense)
r2_dense = r2_score(y_test, y_pred_dense)

print(f'Dense Neural Network - Mean Squared Error: {mse_dense}')
print(f'Dense Neural Network - Mean Absolute Error: {mae_dense}')
print(f'Dense Neural Network - R-squared: {r2_dense}')

# Combine results into a DataFrame for easier comparison
results_dense = pd.DataFrame({
    'Country': countries_2020,
    'Actual Gold': y_test['gold_medals'],
    'Actual Silver': y_test['silver_medals'],
    'Actual Bronze': y_test['bronze_medals'],
    'Predicted Gold': y_pred_dense[:, 0],
    'Predicted Silver': y_pred_dense[:, 1],
    'Predicted Bronze': y_pred_dense[:, 2]
})

# Display the results
print(results_dense)

Epoch 1/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 36.3641 - mae: 2.0716 - val_loss: 9.0881 - val_mae: 1.0430
Epoch 2/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21.5652 - mae: 1.5094 - val_loss: 2.9223 - val_mae: 0.7861
Epoch 3/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.7785 - mae: 1.0818 - val_loss: 2.1326 - val_mae: 0.6075
Epoch 4/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.5096 - mae: 1.2220 - val_loss: 2.2526 - val_mae: 0.6526
Epoch 5/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16.0724 - mae: 1.2440 - val_loss: 2.5189 - val_mae: 0.7092
Epoch 6/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.2050 - mae: 1.1023 - val_loss: 2.6082 - val_mae: 0.7030
Epoch 7/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.4793 - mae: 1.0642 - val_loss: 2.3505 - val_mae: 0.6984
Epoch 8/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.4302 - mae: 1.1421 - val_loss: 2.3916 - val_mae: 0.7014
Epoch 9/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8

# Train CNN

In [52]:
# Reshape data for Conv1D layer
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Build the CNN model
model_cnn = models.Sequential([
    layers.Conv1D(32, 2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(3) # Output layer for gold, silver, and bronze medals
])

# Compile the model
model_cnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history_cnn = model_cnn.fit(X_train_cnn, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)

# Evaluate the model
loss_cnn, mae_cnn = model_cnn.evaluate(X_test_cnn, y_test, verbose=0)

# Make predictions
y_pred_cnn = model_cnn.predict(X_test_cnn)

# Calculate additional metrics
mse_cnn = mean_squared_error(y_test, y_pred_cnn)
mae_cnn = mean_absolute_error(y_test, y_pred_cnn)
r2_cnn = r2_score(y_test, y_pred_cnn)

print(f'CNN - Mean Squared Error: {mse_cnn}')
print(f'CNN - Mean Absolute Error: {mae_cnn}')
print(f'CNN - R-squared: {r2_cnn}')

# Combine results into a DataFrame for easier comparison
results_cnn = pd.DataFrame({
    'Country': countries_2020,
    'Actual Gold': y_test['gold_medals'],
    'Actual Silver': y_test['silver_medals'],
    'Actual Bronze': y_test['bronze_medals'],
    'Predicted Gold': y_pred_cnn[:, 0],
    'Predicted Silver': y_pred_cnn[:, 1],
    'Predicted Bronze': y_pred_cnn[:, 2]
})

# Display the results
print(results_cnn)

Epoch 1/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 43.4189 - mae: 2.1992 - val_loss: 4.2100 - val_mae: 0.7680
Epoch 2/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14.6113 - mae: 1.1946 - val_loss: 2.1021 - val_mae: 0.6458
Epoch 3/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.4025 - mae: 1.2046 - val_loss: 3.0352 - val_mae: 0.7255
Epoch 4/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.0625 - mae: 1.1675 - val_loss: 2.1565 - val_mae: 0.6383
Epoch 5/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.6174 - mae: 1.0091 - val_loss: 2.3137 - val_mae: 0.6481
Epoch 6/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.1473 - mae: 1.0758 - val_loss: 2.3010 - val_mae: 0.6300
Epoch 7/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.5487 - mae: 1.0781 - val_loss: 2.5096 - val_mae: 0.6426
Epoch 8/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.1644 - mae: 1.1009 - val_loss: 2.3130 - val_mae: 0.6289
Epoch 9/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6

# Train RNN

In [53]:
# Reshape data for SimpleRNN layer
X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_rnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Build the RNN model
model_rnn = models.Sequential([
    layers.SimpleRNN(32, activation='relu', input_shape=(X_train_rnn.shape[1], 1)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(3) # Output layer for gold, silver, and bronze medals
])

# Compile the model
model_rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history_rnn = model_rnn.fit(X_train_rnn, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)

# Evaluate the model
loss_rnn, mae_rnn = model_rnn.evaluate(X_test_rnn, y_test, verbose=0)

# Make predictions
y_pred_rnn = model_rnn.predict(X_test_rnn)

# Calculate additional metrics
mse_rnn = mean_squared_error(y_test, y_pred_rnn)
mae_rnn = mean_absolute_error(y_test, y_pred_rnn)
r2_rnn = r2_score(y_test, y_pred_rnn)

print(f'RNN - Mean Squared Error: {mse_rnn}')
print(f'RNN - Mean Absolute Error: {mae_rnn}')
print(f'RNN - R-squared: {r2_rnn}')

# Combine results into a DataFrame for easier comparison
results_rnn = pd.DataFrame({
    'Country': countries_2020,
    'Actual Gold': y_test['gold_medals'],
    'Actual Silver': y_test['silver_medals'],
    'Actual Bronze': y_test['bronze_medals'],
    'Predicted Gold': y_pred_rnn[:, 0],
    'Predicted Silver': y_pred_rnn[:, 1],
    'Predicted Bronze': y_pred_rnn[:, 2]
})

# Display the results
print(results_rnn)

Epoch 1/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 35.3063 - mae: 2.0695 - val_loss: 7.6795 - val_mae: 1.8219
Epoch 2/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.2344 - mae: 1.4400 - val_loss: 2.5727 - val_mae: 0.9006
Epoch 3/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.3339 - mae: 1.2068 - val_loss: 2.5109 - val_mae: 0.8405
Epoch 4/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.6363 - mae: 1.1301 - val_loss: 4.0379 - val_mae: 1.0313
Epoch 5/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.2986 - mae: 1.1704 - val_loss: 2.1859 - val_mae: 0.6002
Epoch 6/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.3884 - mae: 1.0166 - val_loss: 4.9122 - val_mae: 0.9607
Epoch 7/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.5382 - mae: 1.2056 - val_loss: 2.5206 - val_mae: 0.7329
Epoch 8/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.2579 - mae: 1.0969 - val_loss: 4.0662 - val_mae: 0.8908
Epoch 9/130
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.

In [54]:
# Save the DNN model to ../models/olympic_medals_dnn.h5
model_dense.save('../models/olympic_medals_dnn.h5')
model_dense.save('../models/olympic_medals_dnn.keras')